In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
import random
import math
import time
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error
import operator 
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
import datetime
from datetime import date
import operator
from matplotlib import pyplot
#import rpy2 as r
#from tslearn.utils import to_time_series_dataset
#from tslearn.preprocessing import TimeSeriesScalerMinMax
from sklearn import model_selection
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from statsmodels.tsa.stattools import adfuller

In [2]:
#put the files in the same folder with code
confirmed_cases=pd.read_csv('confirmed_cases_total.csv')

In [3]:
deaths_cases=pd.read_csv('deaths_cases_total.csv')

In [4]:
recoveries_cases=pd.read_csv('recoveries_cases_total.csv')

In [5]:
colnames = confirmed_cases.columns.values

In [6]:
colonames = recoveries_cases.columns.values

In [7]:
confirmed = confirmed_cases.loc[:, colnames[4]:colnames[-1]]
deaths = deaths_cases.loc[:, colnames[4]:colnames[-1]]
recoveries = recoveries_cases.loc[:, colonames[4]:colonames[-1]]

In [8]:
time_date = confirmed.columns.values

In [9]:
all_days = np.array([i for i in range(len(time_date))]).reshape(-1, 1)

In [10]:
#pred_days_ahead = 2
#pred_days_ahead = 7
#pred_days_ahead = 30
#predictions_ahead = np.array([i for i in range(len(time_date)+pred_days_ahead)]).reshape(-1, 1)

#predictions_ahead_plus_day = np.array([i for i in range(len(time_date)+pred_days_ahead+1)]).reshape(-1, 1)

In [11]:
pred_days_ahead = 30
predictions_ahead = np.array([i for i in range(len(time_date)+pred_days_ahead)]).reshape(-1, 1)

predictions_ahead_plus_day2 = np.array([i for i in range(len(time_date)+2+1)]).reshape(-1, 1)
predictions_ahead_plus_day7 = np.array([i for i in range(len(time_date)+7+1)]).reshape(-1, 1)
predictions_ahead_plus_day30 = np.array([i for i in range(len(time_date)+30+1)]).reshape(-1, 1)

In [12]:
#start = datetime.datetime.strptime(colnames[4], "%m/%d/%y")
#end = datetime.datetime.strptime(colnames[-1], "%m/%d/%y")
#predictions_ahead_dates = [start + datetime.timedelta(days=d) for d in range(len(predictions_ahead_plus_day))]

start = datetime.datetime.strptime(colnames[4], "%m/%d/%y")
end = datetime.datetime.strptime(colnames[-1], "%m/%d/%y")
predictions_ahead_dates2 = [start + datetime.timedelta(days=d) for d in range(len(predictions_ahead_plus_day2))]
predictions_ahead_dates7 = [start + datetime.timedelta(days=d) for d in range(len(predictions_ahead_plus_day7))]
predictions_ahead_dates30 = [start + datetime.timedelta(days=d) for d in range(len(predictions_ahead_plus_day30))]

In [13]:
confirmed_cases = confirmed_cases[~confirmed_cases['Country/Region'].isin(['Tajikistan', 'Comoros'])]

In [14]:
countries=confirmed_cases['Country/Region'].tolist()
province=confirmed_cases['Province/State'].tolist()

In [15]:
#country = countries[264:-1]
#provinces = province[264:-1]

In [16]:
# def test_stationarity(timeseries):
    
#     #Determing rolling statistics
#     rolmean = timeseries.rolling(12).mean()     
#     rolstd = timeseries.rolling(12).std()
    
#     #Perform Dickey-Fuller test:
#     dftest = adfuller(timeseries, autolag='AIC')
#     dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
#     for key,value in dftest[4].items():
#         dfoutput['Critical Value (%s)'%key] = value
#     return dfoutput

In [17]:
#deaths_cases.tail()

In [18]:
def inverse_difference(last_ob, value):
    return value + last_ob

In [19]:
# #input timeseries as a dataframe
# def make_stationary(timeseries):
#     #remove zeroes to find the optimal d
#     timeseries=timeseries[timeseries['cases'] != 0]
#     #find the optimal d 
#     diff=timeseries-timeseries.shift(1)
#     for d in range(1,10):
#         if (test_stationarity(diff.dropna())['p-value']<0.1):
#             break
#         else: 
#             d=d+1 
#             diff=diff-diff.shift(1)
#     return d 

In [20]:
#df['diff'+str(i)]=df['diff']

In [21]:
def takensEmbedding (data, delay, dimension):
    "This function returns the Takens embedding of data with delay into dimension, delay*dimension must be < len(data)"
    if delay*dimension > len(data):
        raise NameError('Delay times dimension exceed length of data!')    
    embeddedData = np.array([data[0:len(data)-delay*dimension]])
    for i in range(1, dimension):
        embeddedData = np.append(embeddedData, [data[i*delay:len(data) - delay*(dimension - i)]], axis=0)
    return embeddedData;

In [22]:
def prepare_data (region, delay=1 , dimension=31):
    #create a dataframe for timeseries analysis
    date_rng = pd.date_range(start=colnames[4], end=colnames[-1], freq='D') #creates the days
    df = pd.DataFrame(date_rng, columns=['date'])
    df['cases']=np.transpose(region) #this is a dataframe with dates and cases.
#     df = df.set_index('date')
#     timeseries=df[df['cases'] != 0]
#     #find the optimal d 
#     diff=timeseries-timeseries.shift(1)
#     for d in range(1,10):
#         if (test_stationarity(diff.dropna())['p-value']<0.1):
#             break
#         else: 
#             d=d+1 
#             diff=diff-diff.shift(1)
#     for i in range (1,d):
#         i=i+1 
#         diff=diff-diff.shift(1)
#         df['diff'+str(i)]=diff
    #df['diff']=diff #column of the dataframe with the stationary timeseries, might contain Na's!
    df['diff']=df['cases']-df['cases'].shift(1)
    t_diff=(df['diff'].dropna()).to_numpy() 
    timeseries_embedded=np.transpose(takensEmbedding(t_diff,delay,dimension))
    Y_train=timeseries_embedded[:,-1]
    X_train=timeseries_embedded[:,:-1] #keep all apart from the last column
    X_test=t_diff[-(dimension-1):] #predict using as feature the last 'dimension' lags
    return X_train , Y_train, X_test , t_diff

In [23]:
# prepare_data(reg,1,31)

In [24]:
# X_train=prepare_data(reg,1,31)[0]
# Y_train=prepare_data(reg,1,31)[1]
# X_test=prepare_data(reg,1,31)[2]
# t_diff=prepare_data(reg,1,31)[3]

In [25]:
# np.shape(X_train)

In [26]:
# X_test=t_diff[-30:]
# X_test
# pred_days_ahead=2

In [27]:
# predictions=np.zeros(pred_days_ahead)
# for i in range(pred_days_ahead):
#     regressor = RandomForestRegressor(n_estimators = 500, random_state = 42)
#     r=regressor.fit(X_train, Y_train)
#     predictions[i] = regressor.predict(X_test.reshape(1, -1))
#     X_test=np.append(X_test[1:],predictions[i])
#     #in every iteration remove the first element of X_test and add prediction
# predictions

In [28]:
# together=np.append(t_diff,predictions)
# together

In [29]:
# together=np.cumsum(together)
# together

In [30]:
# for i in range(1,d):
#         together=np.cumsum(together)
# together

In [31]:
def predict_with_ML(region, pred_days_ahead, delay=1,dimension=30):
    predictions=np.zeros(pred_days_ahead)
    X_train = prepare_data(region, delay, dimension)[0] 
    Y_train = prepare_data(region, delay, dimension)[1]
    X_test = prepare_data(region, delay, dimension)[2]
    for i in range(pred_days_ahead):
        regressor = RandomForestRegressor(n_estimators = 500, random_state = 42)
        r=regressor.fit(X_train, Y_train)
        predictions[i] = regressor.predict(X_test.reshape(1, -1))
        X_test=np.append(X_test[1:],int(predictions[i])) 
        #in every iteration remove the first element of X_test and add prediction
    predictions #we have to go back to the original scale
    #d = prepare_data(region, delay, dimension)[3]
    t_diff = prepare_data(region, delay, dimension)[3]
    together=np.append(t_diff,predictions)
    #for i in range(1,d):
    together=np.cumsum(together)
    return together #returns an array with all the cases and the predictions (we want the last elements of it)
        

In [32]:
# x=predict_with_ML(reg,30,1,31)

In [33]:
# pred_days_ahead=30
# x[-pred_days_ahead+6]

In [34]:
# region = confirmed_cases.loc[(confirmed_cases['Country/Region'] == "US") & (pd.isnull(confirmed_cases['Province/State']))]
# reg =  np.array(region.loc[:, colnames[4]:colnames[-1]])
# len(reg[0])

In [35]:
# date_rng = pd.date_range(start=colnames[4], end=colnames[-1], freq='D') #creates the days
# df = pd.DataFrame(date_rng, columns=['date'])
# df['cases']=np.transpose(reg) #this is a dataframe with dates and cases.
# df = df.set_index('date')
# #df=df[df['cases'] != 0] #remove zeroes, they do not help for analysis.
# df

In [36]:
# df['diff']=df['cases']-df['cases'].shift(1)
# df

In [37]:
# def takensEmbedding (data, delay, dimension):
#     "This function returns the Takens embedding of data with delay into dimension, delay*dimension must be < len(data)"
#     if delay*dimension > len(data):
#         raise NameError('Delay times dimension exceed length of data!')    
#     embeddedData = np.array([data[0:len(data)-delay*dimension]])
#     for i in range(1, dimension):
#         embeddedData = np.append(embeddedData, [data[i*delay:len(data) - delay*(dimension - i)]], axis=0)
#     return embeddedData;

In [38]:
# t_diff=(df['diff'].dropna()).to_numpy() 
# np.shape(t_diff) #the function wants as input an array

In [39]:
# timeseries_embedded=np.transpose(takensEmbedding(t_diff,1,11))
# np.shape(timeseries_embedded)

In [40]:
# Y_train=timeseries_embedded[:,-1]
# Y_train

In [41]:
# X_train=timeseries_embedded[:,:-1] #keep all apart from the last column
# X_train

In [42]:
#np.transpose(reg)[-30:].reshape(1,-1)
#X_test=np.transpose(x)[0].reshape(1,-1)
#X_test=np.transpose(t_diff[-30:]).reshape(1,-1)
# X_test=t_diff[-10:]
# X_test

In [43]:
# #regressor = RandomForestRegressor(n_estimators = 500, random_state = 42) 
# regressor =  GradientBoostingRegressor(n_estimators=500, learning_rate=0.1,
#                                             max_depth=1, random_state=0, loss='huber')
# #regressor = SVR(kernel='rbf', C=1 )
# r=regressor.fit(X_train, Y_train)
# predictions = regressor.predict(X_test.reshape(1, -1))

In [44]:
# predictions
# #X_test

In [45]:
# X_test=np.append(X_test[1:],int(predictions))
# #len(X_test)
# X_test

In [46]:
# pred_days_ahead=2
# X_train
# #X_train[:-1,:] 

In [47]:
# predictions=np.zeros(pred_days_ahead)
# for i in range(pred_days_ahead):
#     regressor = RandomForestRegressor(n_estimators = 500, random_state = 42)
#     r=regressor.fit(X_train, Y_train)
#     predictions[i] = regressor.predict(X_test.reshape(1, -1))
#     X_test=np.append(X_test[1:],int(predictions[i])) 
#     #in every iteration remove the first element of X_test and add prediction
# predictions

In [48]:
# predictions=np.zeros(pred_days_ahead)
# for i in range(pred_days_ahead):
#     regressor = RandomForestRegressor(n_estimators = 500, random_state = 42)
#     r=regressor.fit(X_train, Y_train)
#     predictions[i] = regressor.predict(X_test)
#     Y_train = Y_train[:-1] #remove the last element of Y_train
#     X_train=X_train[:-1,:] #remove the last row of X_train
# predictions

In [49]:
import operator
def subpopulation_Analysis(cases,country,province, all_days,subpopulation, pred_days_ahead=30):
    
    pred  = list()
    if pd.isnull(province):
        region = cases.loc[(cases['Country/Region'] == country) & (pd.isnull(cases['Province/State']))]
    else:
        print("Hi")
        print(country)
        print(province)
        region = cases.loc[(cases['Country/Region'] == country)& (cases['Province/State'] == province)]
    print("Region")
    print(region)
    
    reg =  np.array(region.loc[:, colnames[4]:colnames[-1]])
    region =  np.array(region.loc[:, colnames[4]:colnames[-1]]).reshape(-1, 1)
        
    #train_x, test_x, train_y, test_y = model_selection.train_test_split(all_days,region,test_size = 0.04,random_state = 42,shuffle=False)
    
    #pred_svm =  SVM_regression(train_x, train_y.ravel(), test_x, test_y, predictions_ahead,subpopulation,all_days,region.ravel())
    #pred.append(pred_svm)
    
    #TODO: a for loop
    daily_increase_threshold1=np.absolute((reg[-1][-1]-reg[-1][-2])/reg[-1][-2])*100
    daily_increase_threshold2=np.absolute((reg[-1][-2]-reg[-1][-3])/reg[-1][-3])*100
    daily_increase_threshold3=np.absolute((reg[-1][-3]-reg[-1][-4])/reg[-1][-4])*100
    daily_increase_threshold4=np.absolute((reg[-1][-4]-reg[-1][-5])/reg[-1][-5])*100
    daily_increase_bound=1000
    daily_increase_threshold = max(daily_increase_threshold1,daily_increase_threshold2,daily_increase_threshold3,daily_increase_threshold4)
    
    if (daily_increase_threshold>daily_increase_bound):
        best_pred_ahead2 = 0 
        best_pred_ahead7 = 0 
        best_pred_ahead30 = 0 
        region = region
        best_pred = 0
        #best_conf_int = np.zeros(2)
        reject_country = 1
        return(best_pred_ahead2,best_pred_ahead7,best_pred_ahead30, region, best_pred, reject_country)
     
    
    pred_arima=predict_with_ML(reg, pred_days_ahead, delay=1,dimension=31)
    #pred.append(pred_svm)
    #pred_RF = RF_regression(train_x, train_y.ravel(), test_x, test_y, predictions_ahead,subpopulation,all_days,region.ravel())
    #pred.append(pred_RF)

    
   # pred_gb =  Grad_Boost_regression(train_x, train_y.ravel(), test_x, test_y, predictions_ahead,subpopulation,all_days,region.ravel())
   # pred.append(pred_gb)


    if (subpopulation == 'confirmed'): 
        if (len(pred_arima)==0):
            min_idx, (min_key, min_val) = min(enumerate(prediction_models_confirmed.items()), key=lambda x: x[1][1])
            best_pred = pred[min_idx]
            #conf_int=np.zeros(2)
        else:
            best_pred=pred_arima
            #conf_int=conf_int
    elif (subpopulation == 'deaths'):
        if (len(pred_arima)==0):
            min_idx, (min_key, min_val) = min(enumerate(prediction_models_deaths.items()), key=lambda x: x[1][1])
            best_pred = pred[min_idx]
            #conf_int=np.zeros(2)
        else:
            best_pred=pred_arima
            #conf_int=conf_int
    else :
        if (len(pred_arima)==0):
            min_idx, (min_key, min_val) = min(enumerate(prediction_models_recoveries.items()), key=lambda x: x[1][1])
            best_pred = pred[min_idx]
            #conf_int=np.zeros(2)
        else:
            best_pred=pred_arima
            #conf_int=conf_int
        
    #print(best_pred)
    #print(type(best_pred))
    best_pred_ahead2 = best_pred[-pred_days_ahead+1]
    #print("gntm2")
    #print(best_pred_ahead2)
    best_pred_ahead7= best_pred[-pred_days_ahead+6]
    #print("gntm7")
    #print(best_pred_ahead7)
    best_pred_ahead30= best_pred[-pred_days_ahead-1]
    #print("gntm30")
    #print(best_pred_ahead30)
    #best_conf_int=conf_int[-1]
   
    #best_conf_int[0]=max(reg[-1][-1], best_conf_int[0])
    #if (best_conf_int[0]>best_conf_int[1]):
        #best_conf_int[1]=2*reg[-1][-1]

    if(best_pred_ahead2<reg[-1][-1]):
        best_pred_ahead2= reg[-1][-1]+2
    
    if(best_pred_ahead7<reg[-1][-1]):
        best_pred_ahead7= reg[-1][-1]+7   
    
    if(best_pred_ahead7<reg[-1][-1]):
        best_pred_ahead7= reg[-1][-1]+30     
    
    reject_country = 0
    
    return best_pred_ahead2,best_pred_ahead7,best_pred_ahead30, region, best_pred, reject_country

In [50]:
pred_file2 = str(2)+"day_prediction_random_forest" +  date.today().strftime('%Y-%m-%d') + ".csv"
pred_file2

'2day_prediction_random_forest2020-06-08.csv'

In [51]:
pred_file7 = str(7)+"day_prediction_random_forest" + date.today().strftime('%Y-%m-%d') + ".csv"
pred_file7

'7day_prediction_random_forest2020-06-08.csv'

In [52]:
pred_file30 = str(30)+"day_prediction_random_forest" + date.today().strftime('%Y-%m-%d') + ".csv"
pred_file30

'30day_prediction_random_forest2020-06-08.csv'

In [53]:
columns = ['Province/State','Country','Target/Date','N','low95N','high95N',
          'R','low95R','high95R','D','low95D','high95D','T','low95T','high95T',
           'M','low95M','high95M','C','low95C','high95C']

            

k = 0

for co in range(len(countries)):
    
    ignore_recoveries = 0    
    
    total_predictions2 = pd.DataFrame(columns = columns)
    total_predictions7 = pd.DataFrame(columns = columns)
    total_predictions30 = pd.DataFrame(columns = columns)
    
    k = k + 1
    

    best_pred_confirmed2,best_pred_confirmed7,best_pred_confirmed30, region_con, best_pred_con, reject_country = subpopulation_Analysis(confirmed_cases,countries[co],province[co],
                                                 all_days,"confirmed",pred_days_ahead)
    
    if (reject_country==1):
        print('rejected_conf')
        continue
        
    best_pred_deaths2,best_pred_deaths7,best_pred_deaths30, region_d, best_pred_d, reject_country = subpopulation_Analysis(deaths_cases,countries[co],province[co],
                                              all_days,"deaths",pred_days_ahead)
    if (reject_country==1):
        print('rejected_d')
        continue
        
    if ((countries[co] == 'US' or countries[co] == 'Canada' ) & (pd.notnull(province[co]))):
        ignore_recoveries=1   
    else:
        best_pred_recoveries2,best_pred_recoveries7,best_pred_recoveries30, region_r, best_pred_r, reject_country = subpopulation_Analysis(recoveries_cases,countries[co],province[co],
                                                  all_days,"recoveries",pred_days_ahead)
    
    if (reject_country==1):
        print('rejected_r')
        continue
    
    # Mortality rate
    #if (conf_int_con[0]==0):
       # mort_pred=0
        #mort_pred_low=0
        #mort_pred_up=0
    #else:
    mort_pred2  = best_pred_deaths2/best_pred_confirmed2
    mort_pred7  = best_pred_deaths7/best_pred_confirmed7
    mort_pred30  = best_pred_deaths30/best_pred_confirmed30
        #mort_pred_low= conf_int_d[0]/conf_int_con[1]
        #mort_pred_up=conf_int_d[1]/conf_int_con[0]
    
    #fix province
    province_name=''
    if (pd.notnull(province[co])) :
        province_name=province[co]
    
    if (ignore_recoveries==0):
        R_value2=int(best_pred_recoveries2) 
        R_value7=int(best_pred_recoveries7) 
        R_value30=int(best_pred_recoveries30) 
        #low_R=int(conf_int_r[0])
        #high_R=int(conf_int_r[1])
    else:
        R_value2=''
        R_value7=''
        R_value30=''
        #low_R=''
        #high_R=''
        
    #Critical Cases:
    #Assume that 5% of all cases are critical and need hospitalization:
    serious_percentage=0.05
    critical2=serious_percentage*best_pred_confirmed2
    critical7=serious_percentage*best_pred_confirmed7
    critical30=serious_percentage*best_pred_confirmed30
    #low_critical=serious_percentage*conf_int_con[0]
    #high_critical=serious_percentage*conf_int_con[1]
    
    total_predictions2 = total_predictions2.append({'Province/State': province_name,
                                                'Country': countries[co],
                                                'Target/Date': predictions_ahead_dates2[-1].strftime('%Y-%m-%d'), 
                                                'N': int(best_pred_confirmed2) ,
                                                'low95N':'',
                                                'high95N': '',
                                                'R': R_value2 ,
                                                'low95R' : '',
                                                'high95R' : '',
                                                'D': int(best_pred_deaths2) ,
                                                'low95D' : '',
                                                'high95D': '',
                                                'T': '',
                                                'low95T' : '' ,
                                                'high95T' : '',
                                                'M': mort_pred2,
                                                'low95M': '',
                                                'high95M': '' ,
                                                'C': int(critical2) ,
                                                'low95C':  '' ,
                                                'high95C': '' },ignore_index = True)
        
    total_predictions7 = total_predictions7.append({'Province/State': province_name,
                                                'Country': countries[co],
                                                'Target/Date': predictions_ahead_dates7[-1].strftime('%Y-%m-%d'), 
                                                'N': int(best_pred_confirmed7) ,
                                                'low95N':'',
                                                'high95N': '',
                                                'R': R_value7 ,
                                                'low95R' : '',
                                                'high95R' : '',
                                                'D': int(best_pred_deaths7) ,
                                                'low95D' : '',
                                                'high95D': '',
                                                'T': '',
                                                'low95T' : '' ,
                                                'high95T' : '',
                                                'M': mort_pred7,
                                                'low95M': '',
                                                'high95M': '' ,
                                                'C': int(critical7) ,
                                                'low95C':  '' ,
                                                'high95C': '' },ignore_index = True)
            
    total_predictions30 = total_predictions30.append({'Province/State': province_name,
                                                'Country': countries[co],
                                                'Target/Date': predictions_ahead_dates30[-1].strftime('%Y-%m-%d'), 
                                                'N': int(best_pred_confirmed30) ,
                                                'low95N':'',
                                                'high95N': '',
                                                'R': R_value30 ,
                                                'low95R' : '',
                                                'high95R' : '',
                                                'D': int(best_pred_deaths30) ,
                                                'low95D' : '',
                                                'high95D': '',
                                                'T': '',
                                                'low95T' : '' ,
                                                'high95T' : '',
                                                'M': mort_pred30,
                                                'low95M': '',
                                                'high95M': '' ,
                                                'C': int(critical30) ,
                                                'low95C':  '' ,
                                                'high95C': '' },ignore_index = True)
    
    if (k==1):
        header = True
    else:
        header = False
    
    total_predictions2.to_csv(pred_file2, sep=',',index = False,  mode='a', header = header)
    total_predictions7.to_csv(pred_file7, sep=',',index = False,  mode='a', header = header)
    total_predictions30.to_csv(pred_file30, sep=',',index = False,  mode='a', header = header)

Region
  Province/State Country/Region  1/22/20  1/23/20  1/24/20  1/25/20  1/26/20  \
0            NaN    Afghanistan        0        0        0        0        0   

   1/27/20  1/28/20  1/29/20  ...  5/29/20  5/30/20  5/31/20  6/1/20  6/2/20  \
0        0        0        0  ...    13659    14525    15205   15750   16509   

   6/3/20  6/4/20  6/5/20  6/6/20  6/7/20  
0   17267   18054   18969   19551   20342  

[1 rows x 140 columns]
Region
  Province/State Country/Region  1/22/20  1/23/20  1/24/20  1/25/20  1/26/20  \
0            NaN    Afghanistan        0        0        0        0        0   

   1/27/20  1/28/20  1/29/20  ...  5/29/20  5/30/20  5/31/20  6/1/20  6/2/20  \
0        0        0        0  ...      246      249      257     265     270   

   6/3/20  6/4/20  6/5/20  6/6/20  6/7/20  
0     294     300     309     327     357  

[1 rows x 140 columns]
Region
  Province/State Country/Region  1/22/20  1/23/20  1/24/20  1/25/20  1/26/20  \
0            NaN    Afghanistan

Region
  Province/State Country/Region  1/22/20  1/23/20  1/24/20  1/25/20  1/26/20  \
6            NaN      Argentina        0        0        0        0        0   

   1/27/20  1/28/20  1/29/20  ...  5/29/20  5/30/20  5/31/20  6/1/20  6/2/20  \
6        0        0        0  ...      520      528      539     556     569   

   6/3/20  6/4/20  6/5/20  6/6/20  6/7/20  
6     583     608     632     648     664  

[1 rows x 140 columns]
Region
  Province/State Country/Region  1/22/20  1/23/20  1/24/20  1/25/20  1/26/20  \
6            NaN      Argentina        0        0        0        0        0   

   1/27/20  1/28/20  1/29/20  ...  5/29/20  5/30/20  5/31/20  6/1/20  6/2/20  \
6        0        0        0  ...     4788     4788     5336    5521    5709   

   6/3/20  6/4/20  6/5/20  6/6/20  6/7/20  
6    5896    5993    6088    6180    6909  

[1 rows x 140 columns]
Region
  Province/State Country/Region  1/22/20  1/23/20  1/24/20  1/25/20  1/26/20  \
7            NaN        Armenia

/home/chrysa/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in long_scalars
/home/chrysa/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in long_scalars
/home/chrysa/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in long_scalars
/home/chrysa/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in long_scalars


Hi
Australia
Northern Territory
Region
        Province/State Country/Region  1/22/20  1/23/20  1/24/20  1/25/20  \
10  Northern Territory      Australia        0        0        0        0   

    1/26/20  1/27/20  1/28/20  1/29/20  ...  5/29/20  5/30/20  5/31/20  \
10        0        0        0        0  ...       29       29       29   

    6/1/20  6/2/20  6/3/20  6/4/20  6/5/20  6/6/20  6/7/20  
10      29      29      29      29      29      29      29  

[1 rows x 140 columns]
Hi
Australia
Queensland
Region
   Province/State Country/Region  1/22/20  1/23/20  1/24/20  1/25/20  1/26/20  \
11     Queensland      Australia        0        0        0        0        0   

    1/27/20  1/28/20  1/29/20  ...  5/29/20  5/30/20  5/31/20  6/1/20  6/2/20  \
11        0        0        1  ...     1058     1058     1058    1059    1059   

    6/3/20  6/4/20  6/5/20  6/6/20  6/7/20  
11    1060    1060    1061    1061    1062  

[1 rows x 140 columns]
Hi
Australia
Queensland
Region
   Provin

Region
   Province/State Country/Region  1/22/20  1/23/20  1/24/20  1/25/20  1/26/20  \
16            NaN        Austria        0        0        0        0        0   

    1/27/20  1/28/20  1/29/20  ...  5/29/20  5/30/20  5/31/20  6/1/20  6/2/20  \
16        0        0        0  ...    15347    15520    15593   15596   15629   

    6/3/20  6/4/20  6/5/20  6/6/20  6/7/20  
16   15672   15717   15742   15789   15793  

[1 rows x 140 columns]
Region
   Province/State Country/Region  1/22/20  1/23/20  1/24/20  1/25/20  1/26/20  \
17            NaN     Azerbaijan        0        0        0        0        0   

    1/27/20  1/28/20  1/29/20  ...  5/29/20  5/30/20  5/31/20  6/1/20  6/2/20  \
17        0        0        0  ...     4989     5246     5494    5662    5935   

    6/3/20  6/4/20  6/5/20  6/6/20  6/7/20  
17    6260    6522    6860    7239    7553  

[1 rows x 140 columns]
Region
   Province/State Country/Region  1/22/20  1/23/20  1/24/20  1/25/20  1/26/20  \
17            NaN 

...

Hi
US
Iowa
Region
    Province/State Country/Region  1/22/20  1/23/20  1/24/20  1/25/20  \
284           Iowa             US        0        0        0        0   

     1/26/20  1/27/20  1/28/20  1/29/20  ...  5/29/20  5/30/20  5/31/20  \
284        0        0        0        0  ...    18957    19244    19552   

     6/1/20  6/2/20  6/3/20  6/4/20  6/5/20  6/6/20  6/7/20  
284   19699   20019   20300   20806   21154   21478   21712  

[1 rows x 140 columns]
Hi
US
Iowa
Region
    Province/State Country/Region  1/22/20  1/23/20  1/24/20  1/25/20  \
284           Iowa             US        0        0        0        0   

     1/26/20  1/27/20  1/28/20  1/29/20  ...  5/29/20  5/30/20  5/31/20  \
284        0        0        0        0  ...      524      531      535   

     6/1/20  6/2/20  6/3/20  6/4/20  6/5/20  6/6/20  6/7/20  
284     555     561     574     583     593     599     605  

[1 rows x 140 columns]
Hi
US
Kansas
Region
    Province/State Country/Region  1/22/20  1/23/20 

Hi
US
Mississippi
Region
    Province/State Country/Region  1/22/20  1/23/20  1/24/20  1/25/20  \
293    Mississippi             US        0        0        0        0   

     1/26/20  1/27/20  1/28/20  1/29/20  ...  5/29/20  5/30/20  5/31/20  \
293        0        0        0        0  ...    14793    15229    15523   

     6/1/20  6/2/20  6/3/20  6/4/20  6/5/20  6/6/20  6/7/20  
293   15752   16041   16322   16560   16769   17039   17270  

[1 rows x 140 columns]
Hi
US
Mississippi
Region
    Province/State Country/Region  1/22/20  1/23/20  1/24/20  1/25/20  \
293    Mississippi             US        0        0        0        0   

     1/26/20  1/27/20  1/28/20  1/29/20  ...  5/29/20  5/30/20  5/31/20  \
293        0        0        0        0  ...      710      723      734   

     6/1/20  6/2/20  6/3/20  6/4/20  6/5/20  6/6/20  6/7/20  
293     739     767     782     794     803     811     817  

[1 rows x 140 columns]
Hi
US
Missouri
Region
    Province/State Country/Region  1

Hi
US
North Carolina
Region
     Province/State Country/Region  1/22/20  1/23/20  1/24/20  1/25/20  \
302  North Carolina             US        0        0        0        0   

     1/26/20  1/27/20  1/28/20  1/29/20  ...  5/29/20  5/30/20  5/31/20  \
302        0        0        0        0  ...    26885    27794    28785   

     6/1/20  6/2/20  6/3/20  6/4/20  6/5/20  6/6/20  6/7/20  
302   29592   30023   31276   32075   33295   34718   35625  

[1 rows x 140 columns]
Hi
US
North Carolina
Region
     Province/State Country/Region  1/22/20  1/23/20  1/24/20  1/25/20  \
302  North Carolina             US        0        0        0        0   

     1/26/20  1/27/20  1/28/20  1/29/20  ...  5/29/20  5/30/20  5/31/20  \
302        0        0        0        0  ...      919      929      937   

     6/1/20  6/2/20  6/3/20  6/4/20  6/5/20  6/6/20  6/7/20  
302     948     961     999    1006    1015    1028    1032  

[1 rows x 140 columns]
Hi
US
North Dakota
Region
    Province/State Cou

Hi
US
South Carolina
Region
     Province/State Country/Region  1/22/20  1/23/20  1/24/20  1/25/20  \
311  South Carolina             US        0        0        0        0   

     1/26/20  1/27/20  1/28/20  1/29/20  ...  5/29/20  5/30/20  5/31/20  \
311        0        0        0        0  ...    11131    11394    11861   

     6/1/20  6/2/20  6/3/20  6/4/20  6/5/20  6/6/20  6/7/20  
311   12148   12415   12415   12415   13453   13453   14286  

[1 rows x 140 columns]
Hi
US
South Carolina
Region
     Province/State Country/Region  1/22/20  1/23/20  1/24/20  1/25/20  \
311  South Carolina             US        0        0        0        0   

     1/26/20  1/27/20  1/28/20  1/29/20  ...  5/29/20  5/30/20  5/31/20  \
311        0        0        0        0  ...      483      487      494   

     6/1/20  6/2/20  6/3/20  6/4/20  6/5/20  6/6/20  6/7/20  
311     500     501     501     501     539     539     546  

[1 rows x 140 columns]
Hi
US
South Dakota
Region
    Province/State Cou

Hi
US
West Virginia
Region
    Province/State Country/Region  1/22/20  1/23/20  1/24/20  1/25/20  \
320  West Virginia             US        0        0        0        0   

     1/26/20  1/27/20  1/28/20  1/29/20  ...  5/29/20  5/30/20  5/31/20  \
320        0        0        0        0  ...     1951     1989     2010   

     6/1/20  6/2/20  6/3/20  6/4/20  6/5/20  6/6/20  6/7/20  
320    2028    2056    2077    2102    2119    2136    2144  

[1 rows x 140 columns]
Hi
US
West Virginia
Region
    Province/State Country/Region  1/22/20  1/23/20  1/24/20  1/25/20  \
320  West Virginia             US        0        0        0        0   

     1/26/20  1/27/20  1/28/20  1/29/20  ...  5/29/20  5/30/20  5/31/20  \
320        0        0        0        0  ...       74       75       75   

     6/1/20  6/2/20  6/3/20  6/4/20  6/5/20  6/6/20  6/7/20  
320      76      78      78      79      84      84      84  

[1 rows x 140 columns]
Hi
US
Wisconsin
Region
    Province/State Country/Regi